In [ ]:
# Install required dependencies
! pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken langchain pypdf faiss-cpu

In [141]:
# Setup openAI connections

import keys

embed_key = keys.embed_key
embed_endpoint = "https://raid-openai-e27bcf212.openai.azure.com/"

gpt_key = keys.gpt_key
gpt_endpoint = "https://raid-ses-openai.openai.azure.com/"

In [ ]:
# some logic here to better split the documents instead of just by page??

In [203]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain

def load_docs(filepath):
    loader = DirectoryLoader(filepath, glob='**/*.pdf', loader_cls=PyPDFLoader)

    docs = loader.load()
    
    return docs

def vector_load(docs, key, endpoint):
    
    embedding_model = OpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_key=key, 
    openai_api_base=endpoint,
    openai_api_version="2023-05-15",
    deployment="text-embedding-ada-002"
    )
    
    # need to build some logic here for checking the database - if exists then just add if not, create
    
    db = FAISS.from_documents(docs, embedding_model)
    
    return db
    

In [135]:
docs = load_docs('./data/')
db = vector_load(docs, embed_key, embed_endpoint)

In [136]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory

# initiate llm
llm = AzureChatOpenAI(openai_api_type="azure", 
                      openai_api_version="2023-05-15", 
                      openai_api_base=gpt_endpoint, 
                      openai_api_key=gpt_key, 
                      deployment_name="raidGPT", 
                      temperature=0.0)

retriever = db.as_retriever(search_kwargs = {"k": 10})

memory = ConversationSummaryMemory(llm=llm,
                                   memory_key="chat_history", 
                                   input_key="question", 
                                   output_key="answer", 
                                   return_messages=True)

qa = ConversationalRetrievalChain.from_llm(llm=llm, 
                                           retriever=retriever, 
                                           return_source_documents=True, 
                                           memory = memory)

In [259]:
FAISS.load_local("dbstore", OpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_key=embed_key, 
    openai_api_base=embed_endpoint,
    openai_api_version="2023-05-15",
    deployment="text-embedding-ada-002"
    ))

In [243]:
result = qa({'question' : "What is a FOEL check?"})

In [244]:
result

{'question': 'What is a FOEL check?',
 'chat_history': [SystemMessage(content='', additional_kwargs={})],
 'answer': "A FOEL check is a routine procedure conducted to ensure that the aircraft systems are operating normally. It should be conducted at regular intervals, with a 10-minute interval suggested as a guide for sortie efficiency. The check involves using the aircraft clock or GPS time on the HUD (Heads-Up Display) to keep track of the time. During in-flight checks, the priority remains flying the aircraft. The FOEL checks are broken up in between the LAPA scan to maintain awareness of the aircraft parameters. For fuel checks, the quantity is called out to the nearest 10 lbs. The checks also include scanning the 'G' recorded on the PFD 'G' meter and checking for the presence of overstress (Red Triangles) on the 'G' meter. When the HUD is available, the max G recorded is displayed on the HUD.",
 'source_documents': [Document(page_content='OFFICIAL (CLOSED)   \n \n10-2 \nOFFICIAL (

In [245]:
qa("What else do I need to check?")

{'question': 'What else do I need to check?',
 'chat_history': [SystemMessage(content="The human asks the AI to explain what a FOEL check is. The AI explains that a FOEL check is a routine procedure conducted to ensure that the aircraft systems are operating normally. It should be conducted at regular intervals, ideally every 10 minutes, and involves using the aircraft clock or GPS time on the Heads-Up Display (HUD) to keep track of time. The checks are broken up between the LAPA scan to maintain awareness of the aircraft parameters, including fuel checks and scanning the 'G' recorded on the PFD 'G' meter. The checks also look for the presence of overstress (Red Triangles) on the 'G' meter. When the HUD is available, the max G recorded is displayed on it.", additional_kwargs={})],
 'answer': "During a FOEL check, in addition to checking the fuel, the aircrew should also scan the 'G' recorded on the PFD 'G' meter. They should also look for the presence of overstress (Red Triangles) on t

In [246]:
qa("What is the maximum number of aircraft that can be in the circuit pattern?")

{'question': 'What is the maximum number of aircraft that can be in the circuit pattern?',
 'chat_history': [SystemMessage(content="The human asks the AI to explain what a FOEL check is. The AI explains that a FOEL check is a routine procedure conducted to ensure that the aircraft systems are operating normally. It should be conducted at regular intervals, ideally every 10 minutes, and involves using the aircraft clock or GPS time on the Heads-Up Display (HUD) to keep track of time. The checks are broken up between the LAPA scan to maintain awareness of the aircraft parameters, including fuel checks and scanning the 'G' recorded on the PFD 'G' meter. The checks also look for the presence of overstress (Red Triangles) on the 'G' meter. When the HUD is available, the max G recorded is displayed on it. The human then asks what else needs to be checked during a FOEL check, and the AI reiterates the importance of checking the fuel, scanning the 'G' meter, looking for overstress, and using t

In [247]:
qa("What are the conditions for a low approach clearance to be given?")

{'question': 'What are the conditions for a low approach clearance to be given?',
 'chat_history': [SystemMessage(content="The human asks the AI to explain what a FOEL check is. The AI explains that a FOEL check is a routine procedure conducted to ensure that the aircraft systems are operating normally. It should be conducted at regular intervals, ideally every 10 minutes, and involves using the aircraft clock or GPS time on the Heads-Up Display (HUD) to keep track of time. The checks are broken up between the LAPA scan to maintain awareness of the aircraft parameters, including fuel checks and scanning the 'G' recorded on the PFD 'G' meter. The checks also look for the presence of overstress (Red Triangles) on the 'G' meter. When the HUD is available, the max G recorded is displayed on it. The human then asks what else needs to be checked during a FOEL check, and the AI reiterates the importance of checking the fuel, scanning the 'G' meter, looking for overstress, and using the HUD wh